<a href="https://colab.research.google.com/github/xxcramseyxx/autoencoder/blob/main/AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.autograd as Variable

In [ ]:
movies = pd.read_csv('/content/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [ ]:
training_set = pd.read_csv('/content/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype=int)
test_set = pd.read_csv('/content/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype=int)


In [ ]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [ ]:
def convert (data):
  new_data=[]
  for id_users in range(1, nb_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [ ]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20,10) #first number is the number of the last layer of nurons, second is current layer of nurons
    self.fc3 = nn.Linear(10,20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return(x)
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = training_set[id_user].unsqueeze(0) #the batch raised to higher dimension unsqueeze 0=value
    target = input.clone() #target is the input vector
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False #make sure we don't calculate the GDescent on target
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()   #direction loss is calculated only used in the training set
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: {} \tloss: {}'.format(epoch, train_loss/s))
  

epoch: 1 	loss: 0.9133884310722351
epoch: 2 	loss: 0.9133444428443909
epoch: 3 	loss: 0.913264274597168
epoch: 4 	loss: 0.9129523634910583
epoch: 5 	loss: 0.9130857586860657
epoch: 6 	loss: 0.9128329157829285
epoch: 7 	loss: 0.9129331111907959
epoch: 8 	loss: 0.9127558469772339
epoch: 9 	loss: 0.9124163389205933
epoch: 10 	loss: 0.9118534922599792
epoch: 11 	loss: 0.9122657775878906
epoch: 12 	loss: 0.9120137095451355
epoch: 13 	loss: 0.9125962853431702
epoch: 14 	loss: 0.9116908311843872
epoch: 15 	loss: 0.9120516777038574
epoch: 16 	loss: 0.9117342829704285
epoch: 17 	loss: 0.9121155738830566
epoch: 18 	loss: 0.9113150238990784
epoch: 19 	loss: 0.9116933345794678
epoch: 20 	loss: 0.9111431837081909
epoch: 21 	loss: 0.9113439321517944
epoch: 22 	loss: 0.910887598991394
epoch: 23 	loss: 0.9109647870063782
epoch: 24 	loss: 0.9106594324111938
epoch: 25 	loss: 0.9107573628425598
epoch: 26 	loss: 0.9103441834449768
epoch: 27 	loss: 0.9109905958175659
epoch: 28 	loss: 0.9105369448661804
epo

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = training_set[id_user].unsqueeze(0) #the batch raised to higher dimension unsqueeze 0=value
    target = test_set[id_user] #target is the input vector
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False #make sure we don't calculate the GDescent on target
      output[(target == 0).unsqueeze(0)] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      test_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
print('loss: {}'.format(test_loss/s))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.9525133967399597
